In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as sp
import scipy.stats as st
import QuantLib as ql
import yfinance as yf
import datetime as dt
# from fredapi import Fred
import quandl
from arch import arch_model

c:\Users\Julia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
start_date = dt.datetime(2024, 1, 1) - dt.timedelta(days=1*365)
end_date = dt.datetime(2024, 1, 1) 
sp500_data = yf.download('^GSPC', start=start_date, end=end_date)
sp500 = sp500_data['Close'].values
S0 = sp500[-1]

quandl.ApiConfig.api_key = '19PYgzG1hHNM7wZyy6NF' 
treasury_history = quandl.get("USTREASURY/YIELD.7", start_date=start_date, end_date=end_date, column_index=1)/100
current_rate = float(treasury_history.iloc[-1].values)

[*********************100%%**********************]  1 of 1 completed

c:\Users\Julia\anaconda3\Lib\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


C:\Users\Julia\AppData\Local\Temp\ipykernel_20256\2096188396.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_rate = float(treasury_history.iloc[-1].values)


In [3]:

# 设定起始和结束日期
start_date = dt.datetime(2024, 1, 1) - dt.timedelta(days=10*365)
end_date = dt.datetime(2024, 1, 1)

# 下载VIX数据
vix = yf.download('^VIX', start=start_date, end=end_date)
vix_data = vix['Close'].values

# 拟合GARCH模型
model = arch_model(vix_data, vol='Garch', p=1, q=1)  # 我们改为GARCH(1,1)模型
result = model.fit()

# 提取模型参数
omega = result.params['omega']
alpha = result.params['alpha[1]']
beta = result.params['beta[1]']

# 定义初始波动率和路径长度
initial_volatility = vix_data[-1]  # 初始波动率
T = 252 * 3  # 三年的交易日数量

# 初始化路径数组
volatility_path = np.zeros(T)
volatility_path[0] = initial_volatility

# 使用GARCH模型参数模拟路径
for t in range(1, T):
    # 使用GARCH方程模拟未来波动率
    volatility_path[t] = np.sqrt(omega + alpha * volatility_path[t-1]**2 + beta * vix_data[-t]**2)


for v in volatility_path:
    print(f"volatility path={v:.2f}")


c:\Users\Julia\anaconda3\Lib\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


[*********************100%%**********************]  1 of 1 completed
Iteration:      1,   Func. Count:      6,   Neg. LLF: 8350607948.4466305
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1655161573.0919487
Iteration:      3,   Func. Count:     19,   Neg. LLF: 36337.3074492383
Iteration:      4,   Func. Count:     25,   Neg. LLF: 26548.86690216142
Iteration:      5,   Func. Count:     31,   Neg. LLF: 7181.730335525123
Iteration:      6,   Func. Count:     37,   Neg. LLF: 202275.36799239612
Iteration:      7,   Func. Count:     43,   Neg. LLF: 6853.853943451664
Iteration:      8,   Func. Count:     48,   Neg. LLF: 6800.925710541648
Iteration:      9,   Func. Count:     53,   Neg. LLF: 39672.31339770514
Iteration:     10,   Func. Count:     59,   Neg. LLF: 6792.864337703582
Iteration:     11,   Func. Count:     64,   Neg. LLF: 6787.406695706052
Iteration:     12,   Func. Count:     69,   Neg. LLF: 6786.831814567824
Iteration:     13,   Func. Count:     74,   Neg. LLF: 6786.786569

In [19]:
# 自定义支付函数类
class CustomPayoff(ql.Payoff):
    def __init__(self, g, G, S, Term, expected_payoff):
        ql.Payoff.__init__(self)
        print("3")
        self.g = g
        self.G = G
        self.S = S
        self.Term = Term
        self.expected_payoff = expected_payoff
        self.K1 = (1+self.g)**(self.Term)*self.expected_payoff  # low strike price
        self.K2 = (1+self.G)**(self.Term)*self.expected_payoff  # high strike price
        self.contract = self.expected_payoff / self.S
        print(f"K1={self.K1:.2f}, K2={self.K2:.2f}, contract={self.contract:.2f}, expected_payoff={self.expected_payoff:.2f}, S={self.S:.2f}, Term={self.Term}, G={self.G:.2f}, g={self.g:.2f}")

    def name(self):
        return "Custom Payoff"

    def __call__(self, price):
        return max(min(self.K2, price * self.contract), self.K1)


# 定义目标函数，用于牛顿法求解
def target_function(G, expected_payoff, g, S, Term):
    # 使用给定的参数构建支付函数
    print("1")
    payoff = CustomPayoff(g, G, S, Term, expected_payoff)
    print("2")
    # 构建QuantLib的日期和日历对象
    calculation_date = ql.Date(1, 1, 2024)
    ql.Settings.instance().evaluationDate = calculation_date
    calendar = ql.NullCalendar()
    day_count = ql.Actual365Fixed()
    # 构建期权对象
    exercise_date = ql.Date(1, 1, 2028)  # 假设期权到期日为2028年1月1日
    exercise = ql.EuropeanExercise(exercise_date)
    print("4")
    option = ql.VanillaOption(payoff, exercise)
    # 构建无风险利率曲线
    risk_free_rate = current_rate  # 当前无风险利率
    flat_curve = ql.FlatForward(calculation_date, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), day_count)
    discount_curve = ql.YieldTermStructureHandle(flat_curve)
    # 计算期权价格
    spot_price = S0  #当前股价
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    process = ql.BlackScholesProcess(spot_handle, discount_curve, ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, 0.0, day_count)))
    engine = ql.AnalyticEuropeanEngine(process)
    option.setPricingEngine(engine)
    option_price = option.NPV()
    # 返回期权价格与期望支付之间的差值，用于求解参数
    return option_price

# 使用牛顿法求解参数
L = 200000  # 期望的期权支付
Term =3
g_list=[0,0.005,0.01,0.015,0.02]
for g in g_list:
    parameter_solution= sp.newton(lambda G: target_function(G, L, g, S0, Term)-L,g)
    # parameter_solution = sp.newton(target_function, initial_guess, args=(expected_payoff,g,S0,Term))
    print("Parameter Solution:", parameter_solution)


1


AttributeError: No constructor defined